In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np

In [65]:
SEARCH_TERM='jordan 1'
PAGE_COUNT=1


#Get top shoe IDs
url = 'https://2fwotdvm2o-dsn.algolia.net/1/indexes/product_variants_v2/query?x-algolia-agent=Algolia for vanilla JavaScript 3.25.1&x-algolia-application-id=2FWOTDVM2O&x-algolia-api-key=ac96de6fef0e02bb95d433d8d5c7038a'

shoeList = pd.DataFrame(columns=['product_template_id', 'name', 'main_picture_url'])

for i in range(PAGE_COUNT):
    data = {'distinct':'true', 'hitsPerPage':'50','page':i, 'query': SEARCH_TERM}
    r = requests.post(url, data=json.dumps(data))
    l = pd.DataFrame.from_records(r.json()['hits'])
    l = l[['product_template_id', 'name', 'main_picture_url']]
    shoeList = shoeList.append(l)
shoeList.rename(columns={'product_template_id': 'id', 'main_picture_url': 'picture'}, inplace=True)
shoeList = shoeList.set_index('id')

In [74]:
shoeList.loc[602201]

name                     Air Jordan 1 Retro High OG 'Patina'
picture    https://image.goat.com/750/attachments/product...
Name: 602201, dtype: object

In [75]:
for id in shoeList.index:
    url = 'https://www.goat.com/web-api/v1/product_variants'
    param = {'productTemplateId':id} 
    header = {'User-Agent': 'Mozilla/5.0'}
    req = requests.get(url, headers=header, params=param)
    jsonInfo = req.json()

    s = pd.DataFrame.from_records(jsonInfo)
    s.loc[s['boxCondition'] == "new_with_defects"]
    s['price'] = s['lowestPriceCents'].apply(lambda x : x['amount']/100)
    
    sizes = s['size'].unique()
    for size in sizes:
        sizeSelection = s.loc[s['size'] == size]
        if len(sizeSelection.loc[(sizeSelection['shoeCondition'] == 'new_no_defects') & (sizeSelection['boxCondition'] == 'good_condition')]):
            if len(sizeSelection.loc[(sizeSelection['shoeCondition'] != 'new_no_defects') | (sizeSelection['boxCondition'] != 'good_condition')]):
                flawless = sizeSelection.loc[(sizeSelection['shoeCondition'] == 'new_no_defects') & (sizeSelection['boxCondition'] == 'good_condition')]
                flawless = flawless['price'].min()
                flawed = sizeSelection.loc[(sizeSelection['shoeCondition'] != 'new_no_defects') | (sizeSelection['boxCondition'] != 'good_condition')]
                flawed = flawed['price'].min()
                if flawless > flawed:
                    print(shoeList.loc[id]['name'],size, flawless - flawed)

Air Jordan 1 Retro High OG 'Patina' 5.5 2.0
Air Jordan 1 Retro High OG 'Patina' 6.0 11.0
Air Jordan 1 Retro High OG 'Patina' 6.5 9.0
Air Jordan 1 Retro High OG 'Patina' 7.0 6.0
Air Jordan 1 Retro High OG 'Patina' 10.0 11.0
Air Jordan 1 Retro High OG 'Patina' 10.5 3.0
Air Jordan 1 Retro High OG 'Patina' 12.0 2.0
Air Jordan 1 Retro High OG 'Hyper Royal' 5.5 18.0
Air Jordan 1 Retro High OG 'Hyper Royal' 8.5 3.0
Air Jordan 1 Retro High OG 'Hyper Royal' 9.5 14.0
Air Jordan 1 Retro High OG 'Hyper Royal' 10.0 35.0
Air Jordan 1 Retro High OG 'Hyper Royal' 10.5 12.0
Air Jordan 1 Retro High OG 'Hyper Royal' 11.5 8.0
Air Jordan 1 Retro High OG 'Hyper Royal' 13.0 15.0
Air Jordan 1 Retro High OG 'Hyper Royal' 17.0 26.0
Air Jordan 1 Retro High OG 'University Blue' 4.0 35.0
Air Jordan 1 Retro High OG 'University Blue' 5.5 9.0
Air Jordan 1 Retro High OG 'University Blue' 6.0 1.0
Air Jordan 1 Retro High OG 'University Blue' 6.5 25.0
Air Jordan 1 Retro High OG 'University Blue' 7.0 7.0
Air Jordan 1 Retr

Air Jordan 1 Mid 'Hyper Royal' 6.5 235.0
Air Jordan 1 Mid 'Hyper Royal' 11.0 1.0
Air Jordan 1 Mid 'Hyper Royal' 15.0 44.0
Air Jordan 1 Retro High OG 'Smoke Grey' 7.5 32.0
Air Jordan 1 Retro High OG 'Smoke Grey' 8.0 17.0
Air Jordan 1 Retro High OG 'Smoke Grey' 8.5 48.0
Air Jordan 1 Retro High OG 'Smoke Grey' 9.0 20.0
Air Jordan 1 Retro High OG 'Smoke Grey' 9.5 39.0
Air Jordan 1 Retro High OG 'Smoke Grey' 10.0 25.0
Air Jordan 1 Retro High OG 'Smoke Grey' 10.5 44.0
Air Jordan 1 Retro High OG 'Smoke Grey' 11.0 49.0
Air Jordan 1 Retro High OG 'Smoke Grey' 11.5 25.0
Air Jordan 1 Retro High OG 'Smoke Grey' 12.0 16.0
Air Jordan 1 Retro High OG 'Smoke Grey' 13.0 35.0
Air Jordan 1 Retro High OG 'Smoke Grey' 15.0 50.0
Air Jordan 1 Retro High OG 'Smoke Grey' 17.0 15.0
Air Jordan 1 Mid SE 'All Star 2021' 9.0 20.0
Air Jordan 1 Mid SE 'All Star 2021' 9.5 21.0
Air Jordan 1 Mid SE 'All Star 2021' 10.5 10.0
Air Jordan 1 Mid SE 'All Star 2021' 12.0 5.0
Air Jordan 1 Mid SE 'All Star 2021' 13.0 11.0
Air Jo

Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 7.0 107.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 7.5 120.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 8.0 39.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 9.0 23.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 9.5 88.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 10.0 118.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 10.5 112.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 11.0 125.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 11.5 206.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 12.0 250.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 12.5 89.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 13.0 153.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 14.0 240.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 15.0 100.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3.0' 16.0 301.0
Air Jordan 1 Retro High OG 'Shattered Backboard 3

Air Jordan 1 Retro High OG Premium 'Yin Yang' 7.0 54.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 8.0 341.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 8.5 471.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 9.5 264.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 10.0 231.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 10.5 174.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 11.0 225.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 12.0 200.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 13.0 50.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 14.0 151.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 16.0 1820.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 17.0 69.0
Air Jordan 1 Retro High OG Premium 'Yin Yang' 18.0 49.0


In [59]:
id = 508716
url = 'https://www.goat.com/web-api/v1/product_variants'
param = {'productTemplateId':id} 
header = {'User-Agent': 'Mozilla/5.0'}
req = requests.get(url, headers=header, params=param)
jsonInfo = req.json()

s = pd.DataFrame.from_records(jsonInfo)
s.loc[s['boxCondition'] == "new_with_defects"]
s['price'] = s['lowestPriceCents'].apply(lambda x : x['amount']/100)
s

,size,sizeOption,shoeCondition,boxCondition,lowestPriceCents,instantShipLowestPriceCents,lowestPriceCentsByWarehouse,price
0,5.0,"{'presentation': '5', 'value': 5}",new_with_defects,good_condition,"{'currency': 'USD', 'amount': 42500, 'amountUs...",{'currency': 'USD'},[],425.0
1,5.0,"{'presentation': '5', 'value': 5}",new_no_defects,good_condition,"{'currency': 'USD', 'amount': 38600, 'amountUs...","{'currency': 'USD', 'amount': 41100, 'amountUs...",[],386.0
2,5.5,"{'presentation': '5.5', 'value': 5.5}",new_no_defects,good_condition,"{'currency': 'USD', 'amount': 39900, 'amountUs...","{'currency': 'USD', 'amount': 48900, 'amountUs...",[],399.0
3,5.5,"{'presentation': '5.5', 'value': 5.5}",new_with_defects,good_condition,"{'currency': 'USD', 'amount': 43800, 'amountUs...",{'currency': 'USD'},[],438.0
4,5.5,"{'presentation': '5.5', 'value': 5.5}",used,badly_damaged,"{'currency': 'USD', 'amount': 56000, 'amountUs...",{'currency': 'USD'},[],560.0
5,6.0,"{'presentation': '6', 'value': 6}",new_no_defects,good_condition,"{'currency': 'USD', 'amount': 39300, 'amountUs...","{'currency': 'USD', 'amount': 48500, 'amountUs...",[],393.0
6,6.0,"{'presentation': '6', 'value': 6}",new_with_defects,good_condition,"{'currency': 'USD', 'amount': 43000, 'amountUs...",{'currency': 'USD'},[],430.0
7,6.0,"{'presentation': '6', 'value': 6}",new_no_defects,no_original_box,"{'currency': 'USD', 'amount': 57500, 'amountUs...",{'currency': 'USD'},[],575.0
8,6.0,"{'presentation': '6', 'value': 6}",new_with_defects,no_original_box,"{'currency': 'USD', 'amount': 52500, 'amountUs...",{'currency': 'USD'},[],525.0
9,6.0,"{'presentation': '6', 'value': 6}",used,missing_lid,"{'currency': 'USD', 'amount': 60000, 'amountUs...",{'currency': 'USD'},[],600.0


In [60]:
sizes = s['size'].unique()
for size in sizes:
    sizeSelection = s.loc[s['size'] == size]
    if len(sizeSelection.loc[(sizeSelection['shoeCondition'] == 'new_no_defects') & (sizeSelection['boxCondition'] == 'good_condition')]):
        if len(sizeSelection.loc[(sizeSelection['shoeCondition'] != 'new_no_defects') | (sizeSelection['boxCondition'] != 'good_condition')]):
            flawless = sizeSelection.loc[(sizeSelection['shoeCondition'] == 'new_no_defects') & (sizeSelection['boxCondition'] == 'good_condition')]
            flawless = flawless['price'].min()
            flawed = sizeSelection.loc[(sizeSelection['shoeCondition'] != 'new_no_defects') | (sizeSelection['boxCondition'] != 'good_condition')]
            flawed = flawed['price'].min()
            if flawless > flawed:
                print(size, flawless - flawed)

7.5 19.0
8.0 13.0
8.5 3.0
9.0 25.0
9.5 45.0
10.0 124.0
10.5 90.0
11.5 163.0
12.0 20.0
